##### Step 1: Initialize Workspace & Environment

In [61]:
from azureml.core import Workspace, Model
from azureml.core import Environment

# Initialize workspace
ws = Workspace.from_config()

# Get the trained model from registry
model_name= 'pima_model_SDKv1_02'
model = ws.models[model_name]
print(model.name, 'version', model.version)


# environment name
env_name= 'ENV-SDKv1'

# ---- create enviroment using .yaml file
# python_packages = Environment.from_conda_specification(env_name, '../dependencies/conda.yaml')
# # register the environment
# python_packages.register(workspace=ws)


pima_model_SDKv1_02 version 4


Step 3: Configure & Deploy Model as a Web Service

In [62]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configure the scoring environment
service_env = Environment.get(workspace=ws, name=env_name)
service_env.inferencing_stack_version="latest" # to use latest version

inference_config = InferenceConfig(source_directory='../src/',
                                   entry_script= 'pima_scoreRealtimeEndpoint_SDKv1.py',
                                   environment=service_env
                                   )

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=4)

# Deploy the model as a service
print('Deploying model...')
service_name = "pima-indians-sdkv1-service3"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)
print(service.get_logs())

Deploying model...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-11-10 18:46:25+00:00 Creating Container Registry if not exists.
2023-11-10 18:46:27+00:00 Use the existing image.
2023-11-10 18:46:28+00:00 Generating deployment configuration.
2023-11-10 18:46:29+00:00 Submitting deployment to compute.
2023-11-10 18:46:34+00:00 Checking the status of deployment pima-indians-sdkv1-service3..
2023-11-10 18:48:18+00:00 Checking the status of inference endpoint pima-indians-sdkv1-service3.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
/bin/bash: /azureml-envs/azureml_dde9de8808e90dc357d096bbcd2dcc0e/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_dde9de8808e90dc357d096bbcd2dcc0e/lib/libtinfo.so.6: no version information available (required by /bin/bash)

/tmp/ipykernel_55961/4153428433.py:20: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)


**Step 4: Test the Web Service**

With the service deployed, now you can consume it from a client application.

In [64]:
import json

x_new = [[0.0, 139.0, 62.0, 17.0, 210.0, 22.1, 0.207, 21.0], [0.0, 139.0, 62.0, 17.0, 210.0, 22.1, 0.207, 21.0]]

# print ('data: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# laoding Json file
# with open('../../data/sample_request.json', 'r') as f:
#   input_json = json.load(f)

# print(input_json)


# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)
predictions


[0, 0]

In [65]:
input_json

'{"data": [[0.0, 139.0, 62.0, 17.0, 210.0, 22.1, 0.207, 21.0], [0.0, 139.0, 62.0, 17.0, 210.0, 22.1, 0.207, 21.0]]}'


You've deployed your web service as an Azure Container Instance (ACI) service that requires no authentication. This is fine for development and testing, but for production you should consider deploying to an Azure Kubernetes Service (AKS) cluster and enabling token-based authentication. This would require REST requests to include an Authorization header.

**Step 5: Delete the service**

When you no longer need your service, you should delete it to avoid incurring unecessary charges.

In [58]:
service.delete()
print ('Service deleted.')

Service deleted.


**Step 6: Deploy the model as a web service to Azuer Kubernetes Cluster**

In [59]:
# # provision aks cluster
# from azureml.core.compute import ComputeTarget
# from azureml.core.compute_target import ComputeTargetException
# from azureml.core.compute import AksCompute, ComputeTarget
# from azureml.core.webservice import Webservice, AksWebservice

# # Choose a name for your AKS cluster
# aks_name = 'my-aks-9' 

# # Verify that cluster does not exist already
# try:
#     aks_target = ComputeTarget(workspace=ws, name=aks_name)
#     print('Found existing cluster, use it.')
# except ComputeTargetException:
#     # Use the default configuration (can also provide parameters to customize)
#     prov_config = AksCompute.provisioning_configuration()

#     # Create the cluster
#     aks_target = ComputeTarget.create(workspace = ws, 
#                                     name = aks_name, 
#                                     provisioning_configuration = prov_config)

# if aks_target.get_status() != "Succeeded":
#     aks_target.wait_for_completion(show_output=True)

In [60]:
# attah the existing cluster if any
# resource_id = '/subscriptions/92c76a2f-0e1c-4216-b65e-abf7a3f34c1e/resourcegroups/raymondsdk0604/providers/Microsoft.ContainerService/managedClusters/my-aks-0605d37425356b7d01'

# create_name='my-existing-aks' 
# # Create the cluster
# attach_config = AksCompute.attach_configuration(resource_id=resource_id)
# aks_target = ComputeTarget.attach(workspace=ws, name=create_name, attach_configuration=attach_config)
# # Wait for the operation to complete
# aks_target.wait_for_completion(True)

In [ ]:
# Deploy webservice to AKS
# Set the web service configuration (using default here)
# aks_config = AksWebservice.deploy_configuration()

# # Enable token auth and disable (key) auth on the webservice
# aks_config = AksWebservice.deploy_configuration(token_auth_enabled=True, auth_enabled=False)

In [ ]:
# aks_service_name ='aks-service-1'

# aks_service = Model.deploy(workspace=ws,
#                            name=aks_service_name,
#                            models=[model],
#                            inference_config=inf_config,
#                            deployment_config=aks_config,
#                            deployment_target=aks_target)

# aks_service.wait_for_deployment(show_output = True)
# print(aks_service.state)

In [ ]:
# Test the Servie
# import json

# test_sample = json.dumps({'data': [
#     [1,2,3,4,5,6,7,8,9,10], 
#     [10,9,8,7,6,5,4,3,2,1]
# ]})
# test_sample = bytes(test_sample,encoding = 'utf8')

# prediction = aks_service.run(input_data = test_sample)
# print(prediction)